In [8]:
import pandas as pd
import numpy as np

df = pd.read_csv('EV_Predictive_Maintenance_Dataset_15min.csv')

In [9]:
# Drop columns you don’t want
feature_cols = [
    'SoC', 'SoH', 'Battery_Voltage', 'Battery_Current',
    'Battery_Temperature', 'Charge_Cycles',
    'Motor_Temperature', 'Motor_Vibration', 'Motor_Torque', 'Motor_RPM',
    'Power_Consumption', 'Brake_Pad_Wear', 'Brake_Pressure', 'Reg_Brake_Efficiency',
    'Tire_Pressure', 'Tire_Temperature', 'Suspension_Load',
    'Ambient_Temperature', 'Ambient_Humidity', 'Load_Weight',
    'Driving_Speed', 'Distance_Traveled', 'Idle_Time', 'Route_Roughness','Maintenance_Type'
]

target_cols = ['RUL', 'TTF', 'Component_Health_Score', 'Failure_Probability']

X = df[feature_cols].values
Y = df[target_cols].values

print(X.shape, Y.shape)


(175393, 25) (175393, 4)


In [10]:
from sklearn.preprocessing import StandardScaler

X_scaler = StandardScaler()
Y_scaler = StandardScaler()

X_scaled = X_scaler.fit_transform(X)
Y_scaled = Y_scaler.fit_transform(Y[:, :3])  # Only the 3 regression targets

# Keep Failure Probability as is (0/1)
Y_scaled = np.hstack([Y_scaled, Y[:, 3].reshape(-1, 1)])

print(X_scaled.shape, Y_scaled.shape)


(175393, 25) (175393, 4)


In [11]:
print(Y_scaled[:10]) # Checking if Failure Probability is scaled correctly

[[ 5.21751612e-01 -3.37218271e-01  4.08673404e-01  0.00000000e+00]
 [-4.19922371e-02  8.97190462e-01  3.13855080e-01  0.00000000e+00]
 [ 6.74139594e-01  7.63501215e-01  4.99769564e-01  0.00000000e+00]
 [ 1.55355804e-01  6.43321357e-01  2.71118254e-01  0.00000000e+00]
 [ 4.83915065e-01  8.54804960e-01 -6.71834788e-04  1.00000000e+00]
 [ 3.49730365e-01  6.62945480e-01 -4.37846588e-02  0.00000000e+00]
 [ 8.27780167e-01  8.85613011e-01  3.61476237e-01  0.00000000e+00]
 [-1.70099960e-01 -5.38288215e-02  6.10019367e-01  0.00000000e+00]
 [-1.61327106e-01 -3.38544571e-01  3.43009879e-01  1.00000000e+00]
 [ 9.88584189e-01  6.26543831e-01  8.69991339e-02  0.00000000e+00]]


In [12]:
import numpy as np

window_size = 12  # 12 x 15 min = 3 hours

X_seq = []
Y_seq = []

for i in range(len(X_scaled) - window_size):
    X_seq.append(X_scaled[i:i+window_size])
    Y_seq.append(Y_scaled[i+window_size])  # Predict next point

X_seq = np.array(X_seq)
Y_seq = np.array(Y_seq)

print(X_seq.shape, Y_seq.shape)


(175381, 12, 25) (175381, 4)


In [13]:
train_size = int(0.7 * len(X_seq))
val_size = int(0.15 * len(X_seq))

X_train, Y_train = X_seq[:train_size], Y_seq[:train_size]
X_val, Y_val = X_seq[train_size:train_size+val_size], Y_seq[train_size:train_size+val_size]
X_test, Y_test = X_seq[train_size+val_size:], Y_seq[train_size+val_size:]

print(X_train.shape, X_val.shape, X_test.shape)

(122766, 12, 25) (26307, 12, 25) (26308, 12, 25)


In [22]:

Y_reg_train = Y_train[:, :3]
Y_class_train = Y_train[:, 3]

Y_reg_val = Y_val[:, :3]
Y_class_val = Y_val[:, 3]

Y_reg_test = Y_test[:, :3]
Y_class_test = Y_test[:, 3]

In [38]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, BatchNormalization,  Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy


In [39]:
#Inputs 
time_steps = 12
num_features = X_train.shape[2]

inputs = Input(shape=(time_steps, num_features))

In [40]:

inputs = Input(shape=(X_train.shape[1], X_train.shape[2]))

# 3 stacked Bidirectional LSTM layers with dropout
x = Bidirectional(LSTM(256, return_sequences=True))(inputs)
x = Dropout(0.3)(x)
x = Bidirectional(LSTM(128, return_sequences=True))(x)
x = Dropout(0.3)(x)
x = Bidirectional(LSTM(64))(x)
x = Dropout(0.3)(x)

# Shared dense layers
x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.3)(x)

# Branch: Regression (3 outputs)
regression_output = Dense(3, activation='linear', name='regression')(x)

# Branch: Classification (binary)
classification_output = Dense(1, activation='sigmoid', name='classification')(x)

model = Model(inputs=inputs, outputs=[regression_output, classification_output])

# Adam optimizer with smaller learning rate
optimizer = Adam(learning_rate=0.0005)

model.compile(
    optimizer=optimizer,
    loss={
        'regression': 'mse',
        'classification': 'binary_crossentropy'
    },
    loss_weights={
        'regression': 1.0,
        'classification': 0.5  # Adjust if needed
    },
    metrics={
        'regression': ['mae'],
        'classification': ['accuracy']
    }
)

model.summary()


Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_7       │ (None, 12, 25)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_3     │ (None, 12, 512)   │    577,536 │ input_layer_7[0]… │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_12          │ (None, 12, 512)   │          0 │ bidirectional_3[… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_4     │ (None, 12, 256)   │    656,384 │ dropout_12[0][0]  │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_13          │ (None, 12, 256)   │          0 │ bidirectional_4[… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_5     │ (None, 128)       │    164,352 │ dropout_13[0][0]  │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_14          │ (None, 128)       │          0 │ bidirectional_5[… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 128)       │     16,512 │ dropout_14[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_15          │ (None, 128)       │          0 │ dense_6[0][0]     │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_7 (Dense)     │ (None, 64)        │      8,256 │ dropout_15[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_16          │ (None, 64)        │          0 │ dense_7[0][0]     │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ regression (Dense)  │ (None, 3)         │        195 │ dropout_16[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ classification      │ (None, 1)         │         65 │ dropout_16[0][0]  │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,423,300 (5.43 MB)

 Trainable params: 1,423,300 (5.43 MB)

 Non-trainable params: 0 (0.00 B)

In [41]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)

history = model.fit(
    X_train,
    {
        'regression': Y_reg_train, 
        'classification': Y_class_train
    },
    validation_data=(
        X_val,
        {
            'regression': Y_reg_val, 
            'classification': Y_class_val
        }
    ),
    epochs=50,
    batch_size=64,
    callbacks=[early_stop, checkpoint],
    verbose=1
)


Epoch 1/50
1919/1919 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step - classification_accuracy: 0.8928 - classification_loss: 0.3703 - loss: 1.1983 - regression_loss: 1.0132 - regression_mae: 0.7274

1919/1919 ━━━━━━━━━━━━━━━━━━━━ 213s 108ms/step - classification_accuracy: 0.8928 - classification_loss: 0.3702 - loss: 1.1983 - regression_loss: 1.0132 - regression_mae: 0.7274 - val_classification_accuracy: 0.8987 - val_classification_loss: 0.3277 - val_loss: 1.1635 - val_regression_loss: 0.9975 - val_regression_mae: 0.7208
Epoch 2/50
1919/1919 ━━━━━━━━━━━━━━━━━━━━ 205s 107ms/step - classification_accuracy: 0.9008 - classification_loss: 0.3309 - loss: 1.1718 - regression_loss: 1.0063 - regression_mae: 0.7241 - val_classification_accuracy: 0.8987 - val_classification_loss: 0.3279 - val_loss: 1.1637 - val_regression_loss: 0.9976 - val_regression_mae: 0.7212
Epoch 3/50
1919/1919 ━━━━━━━━━━━━━━━━━━━━ 227s 118ms/step - classification_accuracy: 0.9018 - classification_loss: 0.3256 - loss: 1.1619 - regression_loss: 0.9990 - regression_mae: 0.7203 - val_classification_accuracy: 0.8987 - val_classification_loss: 0.3275 - val_loss: 1.1635 - val_regression_loss: 0.9976 - val_regression_mae: 0.72